# Feelpp NIRB Online

# Import useful modules 

In [ ]:
import os
### uncomment above two line ones to add the feelpp path with nirb module in the python environement
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
import feelpp
from feelpp.mor.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
from feelpp.mor.nirb.nirb_perf import computeErrorSampling, computeError
import pandas as pd

## Set configuration

In [ ]:
import IPython
module_locals = IPython.extract_module_locals()[1]
# get directory of the notebook, only working if the notebook is run in vscode
if "__vsc_ipynb_file__" in module_locals:
    model_dir = "/".join(module_locals["__vsc_ipynb_file__"].split("/")[:-1]) + "/model"
# if not, we need to hard code the path...
else:
    model_dir = "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model"

Choose config file

In [ ]:
config_file = os.path.join(model_dir, "square/square.cfg")

Set feelpp environnement 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of basis functions saved. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap = config_nirb['nbSnapshots']
nbSnap

Set the input/output path 

In [ ]:
doGreedy = config_nirb['doGreedy']
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

RESPATH = f"results/{rectPath}/{greedyPath}"

Initialize nirb offline class 

In [ ]:
nirb_on = nirbOnline(**config_nirb)
nirb_on.initModel()

## Load offline data

In [ ]:
RESPATH = "/home/saigre/Documents/DELETE_ME/nirb/heat/np_1/results/Rect/noGreedy"
RESPATH

In [ ]:
err = nirb_on.loadData(path=RESPATH)
assert err == 0, "Error while loading data"

In [ ]:
print(f"[NIRB Online] nirb_on has {nirb_on.N} basis vectors")

Choose the parameter to cumpute nirb approximate solution 

In [ ]:
mu = nirb_on.Dmu.element() # get the min value of parameters given in the model
mu.setParameters([0.1, 0.1, 0.5, 0.1])
mu.view()

Compute approximate solution 

In [12]:
if feelpp.Environment.isMasterRank():
    print(f"[NIRB] Online part Done !")

[NIRB] Online part Done !


# NIRB perfs 

uHh = nirb_on.getOnlineSolution(mu, doRectification=False)
uHhR = nirb_on.getOnlineSolution(mu, doRectification=True)
uh = nirb_on.getFineSolution(mu)

In [ ]:
l2Mat = nirb_on.generateOperators()

In [ ]:
print(f"{'Error':>12} {'w/o rect':<20} {'w/ rect':<20}")
print(f"{'Error Xh':>12} {nirb_on.normMat(uHh - uh, l2Mat):<20} {nirb_on.normMat(uHhR - uh, l2Mat):<20}", )
print(f"{'Error L_inf':>12} {(uHh - uh).linftyNorm():<20} {(uHhR - uh).linftyNorm():<20}", )

In [ ]:
nirb_on.initExporter("NirbOnline1param")
nirb_on.exportField(uHh, "uHh")
nirb_on.exportField(uHhR, "uhR")
nirb_on.exportField(uh, "uh")
nirb_on.saveExporter()

Get solution for different size of subspaces

In [ ]:
solutions = []
for n in range(1, nirb_on.N+1):
    solutions.append(nirb_on.getOnlineSolution(mu, n))

In [ ]:
nirb_on.initExporter("NirbN")
nirb_on.exportField(uh, "uh")
for i, uHnN in enumerate(solutions):
    nirb_on.exportField(uHnN, f"uHn{i+1:02}")
    nirb_on.exportField(uh - uHnN, f"diff{i+1:02}")
nirb_on.saveExporter()